In [1]:
import pandas as pd

In [5]:
from urllib import response
from dotenv import load_dotenv
load_dotenv() #Loads all envirnment variables
import os
import google.generativeai as genai


c:\Users\alank\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#%pip install google-generativeai

In [68]:
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))


In [69]:
model =  genai.GenerativeModel('gemini-pro')
def get_gemini_response(question):
    response = model.generate_content(question)
    return response.text

In [9]:
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [10]:
respons = get_gemini_response("Hi how are you")
respons

'I am well, thank you. How are you doing today?'

In [23]:
df = pd.read_excel('University.xlsx')
df.head(3)

,University,Course Name,QS Ranking,World Ranking,Language,Number of sem,Tusion fees,IELTS,Deadline,Application Portal
0,Universität Stuttgart,CS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Otto-Friedrich-UniversitätBamberg -> Universit...,M. Sc. Applied Computer Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Universität Bayreuth,"Computer Science, Master of Science (M.Sc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#QS Ranking
get_gemini_response("As per you what is the QS ranking of {}. Please only give me the excat number or range, but no text".format(df.at[0,'University']))


'156'

In [19]:
#World Ranking
get_gemini_response("As per you what is the world ranking of course {} from {} university. Please try only give me the excat number or range. Also suggest from where data was considered.".format(df.at[1,'Course Name'],df.at[1,'University']))


'The QS World University Rankings by Subject 2023 does not rank the M. Sc. Applied Computer Science program from Otto-Friedrich-Universität Bamberg specifically.'

In [22]:
#lang
get_gemini_response("As per you what is the language of instrauction of {} couse from {} university. Please tell me if its in english or not. If its in English say 'English'.".format(df.at[1,'Course Name'],df.at[1,'University']))


'English'

In [25]:
#Semister
get_gemini_response("As per you how many semister are there for Masters in {} couse in {} university. Please only give me the excat number or range, but no text".format(df.at[0,'Course Name'],df.at[0,'University']))


'4'

In [27]:
#Tusion Fees
get_gemini_response("Can you tell me how much is the tusion fees for {} course in this {} university. Please only give me the excat number or range in Euros, but no text. If there is no fees then say 'No Fees'".format(df.at[11,'Course Name'],df.at[11,'University']))


'€2,451.07'

In [28]:
#IELTS
get_gemini_response("Can you tell me IELTS score required for {} course in this {} university. Please only give me the excat number or range, but no text.".format(df.at[11,'Course Name'],df.at[11,'University']))


'6.5'

In [32]:
#Application
rs = get_gemini_response("Can you tell me what usually is the Application start date and end date for {} course in this {} university. Please only give me Date/Month for both winter and summer intake, but no text.".format(df.at[1,'Course Name'],df.at[1,'University']))
rs.replace('**','').replace('\n\n*','').replace('\n*',' &').replace('\n\n',' | ')

'**Winter Intake:**\n\n* Start: October/November\n* End: March/April\n\n**Summer Intake:**\n\n* Start: April/May\n* End: August/September'

In [62]:
#Application portal
get_gemini_response("Can you tell me what is the Application portal for international students, for this {} course in this {} university. Like University portal or Uni Assist or anything else. Mention 'uni-Assist' & portal name if there's a possibility that apllication for international students is along uni assist. Please no extra text".format(df.at[1,'Course Name'],df.at[1,'University']))


'Uni-Assist: https://www.uni-assist.de/en/'

In [63]:
df.columns

Index(['University', 'Course Name', 'QS Ranking', 'World Ranking', 'Language',
       'Number of sem', 'Tusion fees', 'IELTS', 'Deadline',
       'Application Portal'],
      dtype='object')

In [70]:
for i in range(len(df)):
    #QS Ranking
    df.at[i,'QS Ranking'] = get_gemini_response("As per you what is the QS ranking of {}. Please only give me the excat number or range, but no text".format(df.at[i,'University']))

    #World Ranking
    df.at[i,'World Ranking'] = get_gemini_response("As per you what is the world ranking of course {} from {} university. Please try only give me the excat number or range. Also suggest from where data was considered.".format(df.at[i,'Course Name'],df.at[i,'University']))
    
    #lang
    df.at[i,'Language'] = get_gemini_response("As per you what is the language of instrauction of {} couse from {} university. Please tell me if its in english or not. If its in English say 'English'.".format(df.at[i,'Course Name'],df.at[i,'University']))

    #Semister
    df.at[i,'Number of sem'] = get_gemini_response("As per you how many semister are there for Masters in {} couse in {} university. Please only give me the excat number or range, but no text".format(df.at[i,'Course Name'],df.at[i,'University']))

    #Tusion Fees
    df.at[i,'Tusion fees'] = get_gemini_response("Can you tell me how much is the tusion fees for {} course in this {} university. Please only give me the excat number or range in Euros, but no text. If there is no fees then say 'No Fees'".format(df.at[i,'Course Name'],df.at[i,'University']))

    #IELTS
    df.at[i,'IELTS'] = get_gemini_response("Can you tell me IELTS score required for {} course in this {} university. Please only give me the excat number or range, but no text.".format(df.at[i,'Course Name'],df.at[i,'University']))

    #Application
    rs = get_gemini_response("Can you tell me what usually is the Application start date and end date for {} course in this {} university. Please only give me Date/Month for both winter and summer intake, but no text.".format(df.at[i,'Course Name'],df.at[i,'University']))
    df.at[i,'Deadline'] = rs.replace('**','').replace('\n\n*','').replace('\n*',' &').replace('\n\n',' | ')
    
    #Application portal
    df.at[i,'Application Portal'] = get_gemini_response("Can you tell me what is the Application portal for international students, for this {} course in this {} university. Like University portal or Uni Assist or anything else. Mention 'uni-Assist' & portal name if there's a possibility that apllication for international students is along uni assist. Please no extra text".format(df.at[i,'Course Name'],df.at[i,'University']))


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [71]:
df.head()

,University,Course Name,QS Ranking,World Ranking,Language,Number of sem,Tusion fees,IELTS,Deadline,Application Portal
0,Universität Stuttgart,CS,139,According to the QS World University Rankings ...,English,4,1500 Euros per semester,6.5,Winter intake:\nApplication start date: 15/07\...,**Uni Assist:** uni-assist.de/en/
1,Otto-Friedrich-UniversitätBamberg -> Universit...,M. Sc. Applied Computer Science,266,I cannot provide a world ranking for M. Sc. Ap...,English,4,No Fees,6.5,Winter Intake: | Application Start: September ...,Uni-Assist
2,Universität Bayreuth,"Computer Science, Master of Science (M.Sc.)",835,According to the QS World University Rankings ...,NaN,NaN,NaN,NaN,NaN,NaN
3,Technische HochschuleDeggendorf,AI & Data Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Friedrich-Alexander-UniversitätErlangen-Nürnberg,AI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df.to_excel('New1.xlsx')